In [40]:
import sys
sys.path.insert(0, '..')
from models.model import GPT, GPTConfig
from models import model_types

In [42]:
dataset='multilingual_wiki_single_client_GPT2'
batch_size=8 
dropout = 0.2
lora_dropout = 0.0
gradient_accumulation_steps=16 
learning_rate=32*3e-5
min_lr=6e-5
weight_decay=0.2
decay_lr=True
device='cuda' 
compile=True 
compute_grad_memory=True 
attention_type='standard' 
mlp_type='moe' 
router_type='standard' 
lin_type='standard' 
lora_rank=0 
lora_alpha=32.0 
expert_num=4 
base_seed_offset=2
noise_type='normal' 
load_balancing=True 
moe_lin_type='standard' 
straight_through=False 
global_routing=True
topk_exp=2 
router_lr_scaling=100.0 
router_depth=1 
load_balancing_lambda=0.01
is_cached=True
is_per_token=False

In [43]:
model_args = dict(
        dropout=dropout,
        lora_rank=lora_rank,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        batch_size=batch_size,
        device=device,
        attention_type=attention_type,
        mlp_type=mlp_type,
        router_type=router_type,
        lin_type=lin_type,
        expert_num=expert_num,
        moe_lin_type=moe_lin_type,
        gating_type=model_types.TOPK,
        is_cached=is_cached,
        noise_type=noise_type,
        load_balancing=load_balancing,
        load_balancing_lambda=load_balancing_lambda,
        topk_exp=topk_exp,
        straight_through=straight_through,
        global_routing=global_routing,
        is_per_token=is_per_token,
        router_lr_scaling=router_lr_scaling,
        n_layer=12,
        n_head=12,
        n_embd=768,
        router_depth=router_depth,
    )

In [44]:
import os
import torch
device = 'cuda'
out_dir = '/scratch/homes/dfan/codes/EC-LLM/ec-llm/out-openwebtext-gpt-global_4exp_FF_top2_1layer_router'
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
checkpoint_model_args = checkpoint['model_args']
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)
state_dict = checkpoint['model']
# fix the keys of the state dictionary :(
# honestly no idea how checkpoints sometimes get this prefix, have to debug more
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
iter_num = checkpoint['iter_num']
best_val_loss = checkpoint['best_val_loss']

number of parameters: 293.84M


In [25]:
from datasets import Dataset

abstract_algebra_ds = Dataset.from_file("/scratch/homes/dfan/datasets/mmlu_dataset/abstract_algebra/test/data-00000-of-00001.arrow")

In [31]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")

In [30]:
for i in range(len(abstract_algebra_ds)):
    print(abstract_algebra_ds[i]['question'])
    break

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.


In [33]:
ids = enc.encode_ordinary(abstract_algebra_ds[0]['question'])

In [46]:
model.eval()
model = model.to(device)
model = torch.compile(model)
x = (torch.tensor(ids, dtype=torch.long, device=device)[None, ...])
y = model.generate(x, 1024, temperature=1, top_k=5)


AttributeError: 'CachedRouter' object has no attribute '_forward'